In [1]:
import pandas as pd

# Get new labeled PSSMs

In [2]:
new_df = (pd.read_csv('/home/pierre/riken/data/riken_data/complete_from_xlsx_v2.tsv', sep='\t')
            .loc[lambda x: x.seq_len >= 50])
old_df = (pd.read_csv('/home/pierre/riken/data/riken_data/complete_from_xlsx.tsv', sep='\t')
            .dropna()
            .loc[lambda x: x.seq_len >= 50]
            .assign(old_index=lambda x: x.index.values))

In [27]:
original_fmt = '/home/pierre/riken/data/psiblast/riken_data/{}_pssm.txt'
new_format = '/home/pierre/riken/data/psiblast/riken_data_v2/{}_pssm.txt'

In [28]:
old_df = old_df[['sequences', 'is_allergenic', 'genre', 'species', 'old_index']]
merged = pd.merge(left=new_df, right=old_df, on=['sequences', 'is_allergenic', 'genre', 'species'], how='left')

print(new_df.shape)
print(merged.shape)

merged.info()

indexes = merged[['old_index', 'prot_index']].values
old_index, prot_index = indexes[:, 0], indexes[:, 1]

from shutil import copyfile

for old_i, prot_i in zip(old_index, prot_index):
    try:
        copyfile(original_fmt.format(int(old_i)), new_format.format(prot_i))
    except:
        print('Issue with ', old_i, prot_i)

# New index / Test

In [3]:
from riken.protein_io.data_op import group_shuffle_indices

new_df = (pd.read_csv('/home/pierre/riken/data/riken_data/complete_from_xlsx_v2.tsv', sep='\t')
            .loc[lambda x: x.seq_len >= 50]
            .set_index('prot_index')
         )

old_df = (pd.read_csv('/home/pierre/riken/data/riken_data/complete_from_xlsx.tsv', sep='\t')
            .dropna()
            .loc[lambda x: x.seq_len >= 50]
            .assign(old_index=lambda x: x.index.values.astype(int)))
train_inds, test_inds = group_shuffle_indices(X=old_df.sequences, y=old_df.is_allergenic, groups=old_df.species)
old_df.loc[:, 'is_train'] = None
old_df.loc[:, 'is_train'].iloc[train_inds] = True
old_df.loc[:, 'is_train'].iloc[test_inds] = False

/home/pierre/.conda/envs/python3/lib/python3.5/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [4]:
train_inds.shape

(11141,)

In [5]:
old_df.is_train.sum()

11141

In [6]:
old_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12466 entries, 0 to 12850
Data columns (total 7 columns):
is_allergenic    12466 non-null bool
sequences        12466 non-null object
genre            12466 non-null object
species          12466 non-null object
seq_len          12466 non-null int64
old_index        12466 non-null int64
is_train         12466 non-null bool
dtypes: bool(2), int64(2), object(3)
memory usage: 608.7+ KB


In [7]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12289 entries, 2 to q05187
Data columns (total 5 columns):
is_allergenic    12289 non-null bool
sequences        12289 non-null object
genre            12287 non-null object
species          12285 non-null object
seq_len          12289 non-null int64
dtypes: bool(1), int64(1), object(3)
memory usage: 492.0+ KB


In [8]:
old_df = old_df[['sequences', 'is_allergenic', 'genre', 'species', 'old_index', 'is_train']]
old_df = old_df.drop_duplicates(['sequences', 'is_allergenic', 'genre', 'species'])
merged = new_df.reset_index().merge(right=old_df, 
#                   left_index=True, 
#                   on=['sequences', 'is_allergenic', 'genre', 'species'], 
                  how='left').set_index('prot_index')

In [91]:
merged[merged.index.duplicated()]

,is_allergenic,sequences,genre,species,seq_len,old_index,is_train
prot_index,,,,,,,


In [95]:
merged.head()

,is_allergenic,sequences,genre,species,seq_len,old_index,is_train
prot_index,,,,,,,
2,True,MKFLLVAALCALVAIGSCKPTREEIKTFEQFKKVFGKVYRNAEEEA...,blomia,tropicalis,333,0.0,True
3,True,EIASQIAQEDQSTCEVSKGDFKTFDRMSFTCSFNKSCNVVVAQDCT...,gallus,gallus,284,1.0,True
100,True,MSWQTYVDDHLMCEIEGNYLTSAAIIGQDGSIWAQSASFPQFKPEE...,actinidia,deliciosa,109,2.0,False
caa32835.1,True,MKCLLLALALTCGAQALIVTQTMKGLDIQKVAGTWYSLAMAASDIS...,bos,taurus,178,3.0,True
caa68720.1,True,MWFLALCLAMSLGWTGAEPHFQPRIIGGRECLKNSQPWQVAVYHNG...,canis,lupus,260,4.0,True


In [94]:
merged.to_csv('/home/pierre/riken/data/riken_data/complete_from_xlsx_v2COMPLETE.tsv', sep='\t')